# Trainer for Graph Generated Computer Vision Models in Keras

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Import Data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# dataset = tf.data.Dataset.from_tensor_slices((x_train[0:8] / 255).astype(np.float32))

In [3]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10, dtype='float32')
y_test = to_categorical(y_test, num_classes=10, dtype='float32')

In [4]:
train_dataset_ = tf.data.Dataset.from_tensor_slices(((x_train / 255.).astype(np.float32), y_train))
test_dataset_ = tf.data.Dataset.from_tensor_slices(((x_test / 255.).astype(np.float32), y_test))

In [5]:
train_dataset_

<DatasetV1Adapter shapes: ((32, 32, 3), (10,)), types: (tf.float32, tf.float32)>

In [6]:
def augment(x,y) -> (tf.Tensor, tf.Tensor):
  x = tf.image.random_flip_left_right(x)
  x = tf.image.random_brightness(x, 0.05)
  x = tf.image.random_contrast(x, 0.7,1.3)
  x = tf.image.random_hue(x, 0.08)
  x = tf.clip_by_value(x, 0., 1.)
  return x, y

In [7]:
def augment2(x,y) -> (tf.Tensor, tf.Tensor):
#   x = tf.image.resize_images(x, (36,36), align_corners=True)
  x = tf.image.per_image_standardization(x)
  x = tf.image.random_flip_left_right(x)
  x = tf.image.pad_to_bounding_box(x,4, 4, 40,40)
  x = tf.image.random_crop(x, (32,32,3))
  return x, y

In [8]:
def standardize(x,y):
    x = tf.image.per_image_standardization(x)
    return x, y

In [9]:
train_dataset = train_dataset_.map(augment2,num_parallel_calls=4).shuffle(buffer_size=1000).batch(128).repeat()
test_dataset = test_dataset_.map(standardize,num_parallel_calls=4).batch(128)

W0724 11:09:37.659806 18076 deprecation.py:323] From c:\users\holcm\miniconda3\lib\site-packages\tensorflow\python\ops\image_ops_impl.py:1514: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
train_dataset

<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.float32)>

## Build Model

In [11]:
from tensorflow.keras.layers import Input, Conv2D, Concatenate, BatchNormalization, LeakyReLU, Add, MaxPooling2D, Flatten, Dense, GlobalMaxPooling2D, GlobalAveragePooling2D, Softmax
from tensorflow.keras.models import Model

In [12]:
def build_model_20190717_090720(num_channels):
    y0 = Input(shape=(32,32,3))
    y1 = Conv2D(1*num_channels, (1,1), padding='same', use_bias=False)(y0)
    y2 = BatchNormalization()(y1)
    y3 = LeakyReLU()(y2)
    y4 = Conv2D(1*num_channels, (3,3), padding='same', use_bias=False)(y3)
    y5 = BatchNormalization()(y4)
    y6 = LeakyReLU()(y5)
    y7 = Conv2D(1*num_channels, (3,3), padding='same', use_bias=False)(y6)
    y8 = BatchNormalization()(y7)
    y9 = Conv2D(1*num_channels, (3,3), padding='same', use_bias=False)(y8)
    y10 = BatchNormalization()(y9)
    y11 = LeakyReLU()(y10)
    y12 = Conv2D(1*num_channels, (3,3), padding='same', use_bias=False)(y11)
    y13 = BatchNormalization()(y12)
    y14 = LeakyReLU()(y13)
    y15 = Conv2D(1*num_channels, (3,3), padding='same', use_bias=False)(y14)
    y16 = BatchNormalization()(y15)
    y17 = LeakyReLU()(y16)
    y18 = MaxPooling2D(2,2,padding='same')(y17)
    y19 = Conv2D(2*num_channels, (1,1), padding='same', use_bias=False)(y18)
    y20 = BatchNormalization()(y19)
    y21 = LeakyReLU()(y20)
    y22 = Conv2D(2*num_channels, (3,3), padding='same', use_bias=False)(y21)
    y23 = BatchNormalization()(y22)
    y24 = LeakyReLU()(y23)
    y25 = Conv2D(2*num_channels, (3,3), padding='same', use_bias=False)(y24)
    y26 = BatchNormalization()(y25)
    y27 = LeakyReLU()(y26)
    y28 = Conv2D(2*num_channels, (3,3), padding='same', use_bias=False)(y24)
    y29 = BatchNormalization()(y28)
    y30 = LeakyReLU()(y29)
    y31 = Concatenate()([y27, y30])
    y32 = Conv2D(4*num_channels, (3,3), padding='same', use_bias=False)(y31)
    y33 = BatchNormalization()(y32)
    y34 = LeakyReLU()(y33)
    y35 = Conv2D(4*num_channels, (3,3), padding='same', use_bias=False)(y34)
    y36 = BatchNormalization()(y35)
    y37 = LeakyReLU()(y36)
    y38 = Add()([y34, y37])
    y39 = Conv2D(8*num_channels, (3,3), (2,2), padding='same', use_bias=False)(y38)
    y40 = BatchNormalization()(y39)
    y41 = LeakyReLU()(y40)
    y42 = Conv2D(8*num_channels, (3,3), padding='same', use_bias=False)(y41)
    y43 = BatchNormalization()(y42)
    y44 = LeakyReLU()(y43)
    y45 = Conv2D(8*num_channels, (1,1), padding='same', use_bias=False)(y44)
    y46 = BatchNormalization()(y45)
    y47 = LeakyReLU()(y46)
    y48 = Conv2D(8*num_channels, (3,3), padding='same', use_bias=False)(y47)
    y49 = BatchNormalization()(y48)
    y50 = LeakyReLU()(y49)
    y51 = Conv2D(8*num_channels, (1,1), padding='same', use_bias=False)(y50)
    y52 = BatchNormalization()(y51)
    y53 = LeakyReLU()(y52)
    y54 = Conv2D(8*num_channels, (3,3), padding='same', use_bias=False)(y53)
    y55 = BatchNormalization()(y54)
    y56 = LeakyReLU()(y55)
    y57 = Conv2D(8*num_channels, (3,3), padding='same', use_bias=False)(y44)
    y58 = BatchNormalization()(y57)
    y59 = LeakyReLU()(y58)
    y60 = Concatenate()([y56, y59])
    y61 = GlobalAveragePooling2D()(y60)
    y62 = Flatten()(y61)
    y63 = Dense(10)(y62)
    y64 =  (y63)
    return Model(inputs=y0, outputs=y64)

In [13]:
m = build_model_20190717_090720(24)

W0724 11:09:44.872019 18076 deprecation.py:506] From c:\users\holcm\miniconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 24)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 32, 32, 24)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [15]:
from tensorflow.keras.utils import plot_model
plot_model(m, to_file='model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [16]:
def schedule_fn(epoch):
  if epoch < 40:
    return 1e-5 + 0.05 * epoch / 40.
  elif epoch < 80:
    return 0.05 - 0.05 * (epoch-40.) / 40.
  else:
    return 1e-5

In [17]:
def schedule_fn2(epoch):
  if epoch < 80:
    return 0.1
  elif epoch < 120:
    return 0.01
  else:
    return 0.001

In [18]:
from tensorflow import keras
lr_schedule_cb = keras.callbacks.LearningRateScheduler(schedule_fn2, verbose=0)

In [19]:
csv_log_file = 'best1b.csv'
csv_logger_cb = keras.callbacks.CSVLogger(csv_log_file)

In [20]:

# initiate RMSprop optimizer
opt = keras.optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9)
loss = lambda y_true, y_pred: tf.losses.softmax_cross_entropy(y_true, y_pred)

# Let's train the model using RMSprop
m.compile(loss=loss,
              optimizer=opt, metrics=['acc'])

W0724 11:10:59.626812 18076 deprecation.py:323] From c:\users\holcm\miniconda3\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
history = m.fit(train_dataset, steps_per_epoch=390, epochs=160, validation_data=test_dataset, callbacks=[lr_schedule_cb, csv_logger_cb], validation_steps=78)

Epoch 1/160
390/390 [==============================] - 25s 63ms/step - loss: 1.7988 - acc: 0.3374 - val_loss: 1.7014 - val_acc: 0.3877
Epoch 2/160
390/390 [==============================] - 19s 48ms/step - loss: 1.4876 - acc: 0.4577 - val_loss: 2.0609 - val_acc: 0.3714
Epoch 3/160
390/390 [==============================] - 19s 48ms/step - loss: 1.2256 - acc: 0.5593 - val_loss: 1.3841 - val_acc: 0.5222
Epoch 4/160
390/390 [==============================] - 19s 48ms/step - loss: 1.0439 - acc: 0.6250 - val_loss: 1.0450 - val_acc: 0.6325
Epoch 5/160
390/390 [==============================] - 19s 48ms/step - loss: 0.9186 - acc: 0.6739 - val_loss: 1.4624 - val_acc: 0.5931
Epoch 6/160
390/390 [==============================] - 19s 48ms/step - loss: 0.8249 - acc: 0.7105 - val_loss: 0.8770 - val_acc: 0.7027
Epoch 7/160
390/390 [==============================] - 19s 48ms/step - loss: 0.7559 - acc: 0.7352 - val_loss: 0.8702 - val_acc: 0.6980
Epoch 8/160
390/390 [==============================] - 